In [ ]:
# Meritt Reynolds,  Feburary 11, 2019
# Develop a Python version of the Marklin Grad-Shafranov solver

In [ ]:
from numpy import *
import matplotlib.pyplot as plt
from scipy import sparse

from gs import *

In [ ]:
a = random.randn(2,2)
(np,ndim) = shape(a)
print(np,ndim)



In [ ]:
class Taylor:
    def __init__(self, psi, lam, rax):
        self.psi = psi
        self.lam = lam
        self.rax = rax

class GS:
    def __init__(self):
        self.ds = None
    def __str__(self):
        return "Grad-Shafranov object"


In [ ]:
class Mesh:
    def __init__(self, r, z):
        self.r = r;
        self.z = z;
        self.nx = shape(r)
    def __str__(self):
        return "Mesh of size %d,%d" % self.nx
    


In [ ]:
rmin = 0
rmax = 2
nr = 29
zmin = 0
zmax = 1
nz = 19

(r,z) = meshgrid(linspace(rmin, rmax, nr), linspace(zmin, zmax, nz), indexing='ij')

# Check that first index is radial-like, as is our convention
print(r[0,0], r[1,0])

# Check that second index is axial-like
print(z[0,0], z[0,1])

assert r[1,0] - r[0,0] > 0

In [ ]:
(X,Q) = gs_quad_version_of_rz_array(r, z)

In [ ]:
i = 0
k = Q[i,:]
print('r z:')
for j in range(4):
    print(X[k[j],:])

In [ ]:
m = Mesh(r, z)
print(m)

In [ ]:
gs = gssetup(m)
print(gs)

In [ ]:
# Visualize matrix structure
plt.figure(figsize=(20,20))
plt.spy(gs.ds,markersize=2)
plt.show()

In [ ]:
# What does the boundary flag array look like
m.bp = reshape(gs.bp, m.nx)
plt.imshow(m.bp)
plt.colorbar()
plt.show()

In [ ]:
plt.plot(gs.ds.diagonal())
plt.show()

In [ ]:
# Calculate Taylor state
gs = gstaylor(gs, verbose=True)


In [ ]:
plt.figure(figsize=(10,10))
plt.contourf(m.r, m.z, reshape(gs.psi, m.nx))
plt.colorbar()
plt.axis('equal')
plt.xlabel('r')
plt.ylabel('z')
plt.title('Taylor state, $\lambda_{fc}=%g$' % gs.taylor.lam)
plt.show()

In [ ]:
# Lambda profile polynomial, rather hollow
# Normalization of this function is irrelevant

cLambda = array((-0.9,1,0))

x = linspace(0, 1, 101)
plt.plot(x, polyval(cLambda, x))
plt.show()

In [ ]:
# Integral of lambda polynomial is F polynomial
# We could set an offset, but for now leave it zero

gs.Fpoly = polyint(cLambda)

# Solve the GS equation
gs.Iternums = 100
gs = gspoly(gs)

In [ ]:
plt.figure(figsize=(10,6))
cs = plt.contour(m.r, m.z, reshape(gs.psi, m.nx), linspace(0, 1, 11), colors='k')
plt.clabel(cs, fmt='%g')
plt.axis('equal')
plt.xlabel('r')
plt.ylabel('z')
plt.title('GS solution, contours of $\psi$')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.contourf(m.r, m.z, reshape(gs.dFdpsi, m.nx), 101)
clb = plt.colorbar()
clb.ax.set_title('$\lambda$', fontsize=20)
plt.axis('equal')
plt.xlabel('r')
plt.ylabel('z')
#plt.title('GS solution, $\lambda_{fc}=%g$' % gs.lam)
plt.show()

In [ ]:
plt.plot(gs.psi, gs.dFdpsi, '.')
plt.xlabel('$\psi$')
plt.ylabel('$dF/d\psi$')
plt.show()

In [ ]:
# The delstar operator is -(r/a)*w
# r*Jphi = -delstar*psi = (r/a)*w*psi
# a*Jphi = w*psi
# current is sum of this, so
# dCurrent = w*psi
# We have sum(dCurrent) = 0
# To get total plasma current we exclude boundary points

dCurrent = gs.ds.dot(gs.psi)
print('total current:', sum(dCurrent))

current = sum(dCurrent[~gs.bp])
print('plasma current:', current)

dCurrent[gs.bp] = 0
plt.imshow(reshape(dCurrent,m.nx))
plt.show()



In [ ]:
m.psi = reshape(gs.psi, m.nx)
plt.plot(m.psi)
plt.show()